네이버 전처리
1. DB 구축을 위한 primary 키 삽입 string(위도, 경도)
2. 중복제거(사업장명, 위도, 경도)
3. primary key nan제거

In [2]:
import pandas as pd
import numpy as np
import re
import os

In [3]:
naver_df = pd.read_csv('../../naver_crw/DAOU/test_large_out_v3.csv', index_col=0, encoding='euc-kr')

In [4]:
print(naver_df.info())
naver_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6664 entries, 0 to 6663
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   사업장명                    6664 non-null   object 
 1   업소유형                    874 non-null    object 
 2   업태구분명                   6664 non-null   object 
 3   지번주소                    6653 non-null   object 
 4   도로명주소                   3986 non-null   object 
 5   가까운 지하철역                5774 non-null   object 
 6   지하철역과의 거리               6664 non-null   float64
 7   도보시간                    6664 non-null   float64
 8   위도                      5774 non-null   float64
 9   경도                      5774 non-null   float64
 10  별점                      310 non-null    float64
 11  방문자 리뷰수                 685 non-null    object 
 12  블로그 리뷰수                 513 non-null    object 
 13  한실수                     5434 non-null   float64
 14  양실수                     5347 non-null   

In [ ]:
naver_df.head()

In [6]:
# 프라이머리 키 생성
naver_df['naver_pk'] = np.nan
for i in range(naver_df.shape[0]):
    if pd.isnull(naver_df['위도'][i]) and pd.isnull(naver_df['경도'][i]):
        naver_df['naver_pk'][i] = np.nan
    else:        
        naver_df['naver_pk'][i] = f"{naver_df['위도'][i]:.10f}, {naver_df['경도'][i]:.10f}"   



C:\Users\tjoeun\AppData\Local\Temp\ipykernel_2400\1959696260.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  naver_df['naver_pk'][i] = f"{naver_df['위도'][i]:.10f}, {naver_df['경도'][i]:.10f}"


In [14]:
# .. 위경도 프라이머리키로 중복 제거
# 중복 개수 확인
df_temp1 = naver_df.groupby(['naver_pk'])['도로명주소'].count().reset_index(name='호실개수')
duplicate_name = df_temp1[df_temp1.호실개수 >= 2] # 21개
duplicate_name
num_nankey = len(naver_df[pd.isna(naver_df['naver_pk'])])
print(f'중복 개수 {len(duplicate_name)}, nan 개수 {num_nankey}')

# 키 중복 제거
naver_df_p1 = naver_df.drop_duplicates(['naver_pk'])    # 6664 --> 6383
print(len(duplicate_name), naver_df_p1.shape[0], naver_df.shape[0])

# 이름으로 중복있는데 하나는 nan이여서 제거 안된것 제거
naver_df_p1.dropna(axis=0, subset=['naver_pk'], inplace=True)

중복 개수 142, nan 개수 890
142 5439 6664


C:\Users\tjoeun\AppData\Local\Temp\ipykernel_2400\4206036427.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  naver_df_p1.dropna(axis=0, subset=['naver_pk'], inplace=True)


In [15]:
naver_df_p1[naver_df_p1['사업장명'] == "삼보장"]

,사업장명,업소유형,업태구분명,지번주소,도로명주소,가까운 지하철역,지하철역과의 거리,도보시간,위도,경도,...,네이버 이런점이 좋아요 {항목:좋아요수},데이터갱신일자,인허가일자,인허가취소일자,영업상태코드,폐업일자,휴업시작일자,휴업종료일자,재개업일자,naver_pk
2269,삼보장,NaN,여관업,서울특별시 동대문구 답십리동 99-9번지 (전답3길16),NaN,"답십리, 5호선",884.759247,13.264756,37.573912,127.057093,...,NaN,2018-08-31 23:59:59.0,19850503,NaN,3,20081021,NaN,NaN,NaN,"37.5739124408, 127.0570928450"


In [16]:
naver_df_p1.to_csv("../DAOU/naver.csv", encoding='euc-kr', index=False)